In [1]:
import pandas as pd
import numpy as np

# Generación de datos aleatorios

In [2]:
df = pd.read_csv("datos_arreglados.csv", sep=";", encoding="latin1")
df = df[df["ID"] > 60] #esto pues las primeras filas estaban falladas

Fijamos la semilla pero se puede eliminar para generar muestras diferentes y no siempre la misma.

In [3]:
np.random.seed(42)

Ahora, fijamos el número de datos que queremos crear

In [4]:
n_sint = 2000

Creamos un dataframe donde tendremos los nuevos datos, estos datos serán generados con diferentes combinaciones de "categorías"

In [5]:
df_sint = pd.DataFrame()

# 2.1. ID nuevo (que no coincida con el original)
df_sint["ID"] = range(1, n_sint + 1)

In [6]:
print(df["registration_started"].head(20))

0     02-05-2025 10:15
1     02-05-2025 10:21
2     02-05-2025 10:30
3     02-05-2025 10:47
4     02-05-2025 10:52
5     02-05-2025 10:54
6     02-05-2025 11:14
7     02-05-2025 11:12
8     02-05-2025 11:46
9     02-05-2025 12:16
10    02-05-2025 12:24
11    02-05-2025 12:30
12    02-05-2025 10:42
13    02-05-2025 12:05
14    02-05-2025 12:44
15    02-05-2025 12:59
16    02-05-2025 13:06
17    02-05-2025 13:57
18    02-05-2025 13:56
19    02-05-2025 13:54
Name: registration_started, dtype: object


Para generar datos más realistas, los tiempos también se generaran al azar

In [7]:
df.columns = df.columns.str.strip()

# Convertimos a datetime las columnas de tiempo
for col in ["registration_started", "registration_finished"]:
    df[col] = pd.to_datetime(df[col], format="%d-%m-%Y %H:%M", errors="coerce")

# Duración real en segundos
duracion_real = (
    df["registration_finished"] - df["registration_started"]
).dt.total_seconds().dropna()

# ----------------- TIEMPOS SINTÉTICOS -----------------
start_base = df["registration_started"].dropna().sample(
    n_sint, replace=True
).reset_index(drop=True)

dur_base = duracion_real.sample(n_sint, replace=True).reset_index(drop=True)

jitter_start = np.random.normal(loc=0, scale=600, size=n_sint)  # ±10 min
jitter_dur   = np.random.normal(loc=0, scale=30,  size=n_sint)  # ±30 s

dur_min = duracion_real.min()
dur_max = duracion_real.max()

dur_sint = dur_base + jitter_dur
dur_sint = np.clip(
    dur_sint,
    a_min=max(1, dur_min * 0.5),
    a_max=dur_max * 1.5
)

start_sint  = start_base + pd.to_timedelta(jitter_start, unit="s")
finish_sint = start_sint + pd.to_timedelta(dur_sint, unit="s")

df_sint["registration_started"]  = start_sint
df_sint["registration_finished"] = finish_sint

Para crear los datos, notemos que los estudiantes de bachelor tienen rellenadas dos columnas que los estudiantes de magister no.

In [8]:
# Quitamos columnas ya tratadas
df_sint["SUMA"] = 6
cols_excluir = ["ID", "registration_started", "registration_finished", "SUMA"]
sin_excluir_cols = [c for c in df.columns if c not in cols_excluir]
select_cols = [col for col in sin_excluir_cols if "Select" in col]
num1_cols = [col for col in sin_excluir_cols if "Select" not in col and "_1" in col]
otras_cols = [col for col in sin_excluir_cols if "Select" not in col and "_1" not in col]

In [9]:
df_bachelor = df[df["program"] == "bachelor"]
df_master = df[df["program"] == "master"]
print("---------------Últimas 2 columnas------------\n")
print(f"Hay {len(df_bachelor)} estudiantes de bachelor y {len(df_master)} estudiantes de master\n")
print("La penúltima columna hay espacios en blanco:\n")
print(f"Bachelor: {df_bachelor[select_cols[-2]].isna().sum()}, Master: {df_master[select_cols[-2]].isna().sum()}")
print("\nLa última columna hay espacios en blanco:\n")
print(f"Bachelor: {df_bachelor[select_cols[-1]].isna().sum()}, Master: {df_master[select_cols[-1]].isna().sum()}")

---------------Últimas 2 columnas------------

Hay 306 estudiantes de bachelor y 468 estudiantes de master

La penúltima columna hay espacios en blanco:

Bachelor: 0, Master: 468

La última columna hay espacios en blanco:

Bachelor: 0, Master: 468


Por ello, importa si son estudiantes de bachelor o master al momento de rellenar las columnas

In [11]:
j = 0
for i in range(len(sin_excluir_cols)):
    col = sin_excluir_cols[i]
    col_data = df[col].dropna()
    col_data_master = df_master[col].dropna()
    col_data_bachelor = df_bachelor[col].dropna()

    if col_data.empty:
        # si la columna está vacía, llenamos con NaN
        df_sint[col] = np.nan
        continue
    
    # Para elegir campus o programa, todos deben tenerlo (i=0 e i=1)
    if i <= 1:
        categorias = col_data.unique()
        df_sint[col] = np.random.choice(categorias, size=n_sint)
        # Para las columnas siguientes todas deben tener hasta 8 elecciones de talleres diferentes
    elif col in select_cols:
        if j == 0:
            df_sint[col] = np.nan * n_sint
            df_sint.loc[df_sint["program"]=="bachelor", col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
            df_sint.loc[df_sint["program"]=="master", col] = np.random.choice(
                df_master[col].dropna().unique(),
                size= (df_sint["program"] == "master").sum(),
                replace=True
            )
        elif j < len(select_cols) - 2:
            categorias_bach = df_bachelor[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "bachelor"]:
                # valores ya usados en esta fila, en las columnas anteriores
                usados = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                # categorías disponibles (las que NO están usadas)
                disponibles = [c for c in categorias_bach if c not in usados]

                if disponibles:
                    df_sint.at[idx, col] = np.random.choice(disponibles)
                else:
                    df_sint.at[idx, col] = np.nan

            categorias_master = df_master[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "master"]:
                usados = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                disponibles = [c for c in categorias_master if c not in usados]

                if disponibles:
                    df_sint.at[idx, col] = np.random.choice(disponibles)
                else:
                    df_sint.at[idx, col] = np.nan

        # En las últimas 2 columnas, solo bachelor tiene programas
        else:
            df_sint[col] = np.nan
            df_sint.loc[(df_sint["program"]=="bachelor"), col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
        j += 1
    elif col in otras_cols:
        df_sint[col] = np.nan
        for a in range(3):
            mask = (df_sint[select_cols[a]].astype(str).str.contains(col, na=False))
            df_sint.loc[mask, col] = str(a + 1)
    else:
        df_sint[col] = np.nan

    

    
    






# Guardar el csv y que coincidan las columnas
df_sint = df_sint[df.columns]
df_sint.to_csv("datos_nuevos.csv", index=False)


C:\Users\tatyf\AppData\Local\Temp\ipykernel_33692\976649091.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Psykologisk tryghed i teams' 'Tværfagligt problemdesign'
 'Motivation, trivsel og work-life balance'
 'SCRUM-teknikker i projektarbejdet'
 'Motivation, well-being and work-life balance?'
 'Digitale praksisser i projektarbejdet' 'Systems thinking in engineering'
 'Tværfagligt problemdesign' 'Interdisciplinary problem design'
 'Systemtænkning i ingeniørvidenskab' 'Digital practices in project work'
 'Facilitering af produktive møder' 'Systemtænkning i ingeniørvidenskab'
 'Psykologisk tryghed i teams' 'Facilitering af produktive møder'
 'Digitale praksisser i projektarbejdet' 'Psykologisk tryghed i teams'
 'SCRUM-teknikker i projektarbejdet' 'Generativ AI i studiepraksis'
 'Motivation, well-being and work-life balance?'
 'Interdisciplinary problem design' 'Generative AI in study practices?'
 'M